In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from constants_and_util import *
import image_processing
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pickle
import train_models
import seaborn as snsage
import non_image_data_processing
from image_processing import PytorchImagesDataset
import analysis
import pandas as pd
from scipy.stats import spearmanr
assert USE_HELD_OUT_TEST_SET

# Load in model and data


In [ ]:

all_results = analysis.load_all_results(binary=False, 
                                        min_timestring='2019_06_20', 
                                        thing_to_filter_by={'experiment_to_run':'train_best_model_continuous', 
                                                             'crop_to_just_the_knee':False})

pd.set_option('precision', 3)
all_results

In [ ]:
ensemble_results, ensemble_test_yhat = analysis.try_ensembling(
    all_results, 5, binary_prediction=False)
ensemble_results

In [ ]:
for i in range(1):
    print("Model %i" % i)
    pytorch_model, dataloaders, datasets, dataset_sizes, yhats, dataset_kwargs = train_models.load_model_and_data_from_timestring(
        all_results.iloc[i]['timestring'],
        compute_yhats=True,
        make_the_cam_plots=True, 
        make_the_prediction_change_plots=True)
    
    assert np.allclose(all_results.iloc[0]['test_r'], 
                   analysis.assess_performance(yhat=yhats['test_yhat'],
                            y=datasets['test'].non_image_data['koos_pain_subscore'].values, 
                            binary_prediction=False)['r'])

    assert np.allclose(all_results.iloc[0]['negative_test_rmse'], 
                   analysis.assess_performance(yhat=yhats['test_yhat'],
                            y=datasets['test'].non_image_data['koos_pain_subscore'].values, 
                            binary_prediction=False)['negative_rmse'])
    


In [ ]:
# Make ensemble prediction change plots. 

rng = random.Random(42)
image_idxs_for_interpretability_plots = rng.sample(range(len(datasets['test'].non_image_data)), 8)
print("Images making ensemble plots for", image_idxs_for_interpretability_plots) # these should be same as for individual models. 

train_models.make_prediction_change_plots(list(all_results.iloc[:5]['timestring'].values), 
                             dataset_kwargs=dataset_kwargs, 
                             n_images_to_plot=len(image_idxs_for_interpretability_plots), 
                             figtitle='prediction_change_ensemble_top_5_models.png', 
                             img_idxs=image_idxs_for_interpretability_plots)

# Make descriptive statistics table. 

In [ ]:
analysis.make_descriptive_stats_table(train_df=datasets['train'].non_image_data, 
                                      val_df=datasets['val'].non_image_data,
                                      test_df=datasets['test'].non_image_data)

In [ ]:
# extract relevant variables. 

dataset_name = 'test'
yhat = ensemble_test_yhat
y = datasets[dataset_name].non_image_data['koos_pain_subscore'].values
klg = datasets[dataset_name].non_image_data['xrkl'].values
all_ses_vars, income_at_least_50k, graduated_college, race_black = analysis.extract_all_ses_vars(datasets[dataset_name].non_image_data)
discretized_yhat = analysis.discretize_yhat_like_kl_grade(yhat_arr=yhat,
                                                          kl_grade_arr=klg,
                                                          y_col='koos_pain_subscore')
ids = datasets[dataset_name].non_image_data['id'].values
decile_yhat = analysis.cut_into_deciles(yhat, y_col='koos_pain_subscore')
binarized_y = binarize_koos(y)


print('y mean median and std', np.mean(y), np.median(y), np.std(y, ddof=1))
print('yhat mean median and std', np.mean(yhat), np.median(yhat), np.std(yhat, ddof=1))
print('KLG mean median and std',  np.mean(klg), np.median(klg), np.std(klg, ddof=1))

for ses_var in all_ses_vars:
    ses_arr = all_ses_vars[ses_var]
    print("disaggregate by %s" % ses_var)
    print('ses_var = 1: mean y %2.3f, mean KLG = %2.3f, mean yhat = %2.3f' % (y[ses_arr == 1].mean(), 
                                                                              klg[ses_arr == 1].mean(), 
                                                                              yhat[ses_arr == 1].mean()))
    print('ses_var = 0: mean y %2.3f, mean KLG = %2.3f, mean yhat = %2.3f' % (y[ses_arr == 0].mean(), 
                                                                              klg[ses_arr == 0].mean(), 
                                                                              yhat[ses_arr == 0].mean()))



# Compare yhat's performance on predicting binarized high pain to KLG's. 

In [ ]:

yhat_binary_performance = analysis.assess_performance(yhat=-ensemble_test_yhat,
                                                      y=binarized_y,
                                                      binary_prediction=True,
                                                      return_tpr_and_fpr=True)
                                                        
yhat_binary_performance['predictor'] = 'yhat'

discretized_yhat_binary_performance = analysis.assess_performance(yhat=discretized_yhat,
                                                                  y=binarized_y,
                                                                  binary_prediction=True, 
                                                                  return_tpr_and_fpr=True)
discretized_yhat_binary_performance['predictor'] = 'discretized_yhat'

klg_binary_performance = analysis.assess_performance(yhat=klg,
                                                     y=binarized_y,
                                                     binary_prediction=True, 
                                                     return_tpr_and_fpr=True)
klg_binary_performance['predictor'] = 'klg'

print(pd.DataFrame([yhat_binary_performance, 
                    discretized_yhat_binary_performance,
                    klg_binary_performance])[['predictor', 'auc', 'auprc']])

# Make TPR/FPR plot. 
plt.figure(figsize=[4, 4])
plt.plot(yhat_binary_performance['fpr'], yhat_binary_performance['tpr'], label='$\hat y$')
plt.plot(discretized_yhat_binary_performance['fpr'], 
         discretized_yhat_binary_performance['tpr'], label='Discretized $\hat y$')
plt.plot(klg_binary_performance['fpr'], klg_binary_performance['tpr'], label='KLG')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.legend()
plt.plot([0, 1], [0, 1], '--', color='black')
plt.savefig('auc.png', dpi=300)

print("Computing CIs on yhat binarized performance compared to KLG")
analysis.bootstrap_CIs_on_model_performance(y=binarized_y,
                                            yhat=-yhat, 
                                            yhat_from_klg=klg, 
                                            yhat_from_clinical_image_features=None,
                                            ids=ids, 
                                            binary_prediction=True,
                                            n_bootstraps=N_BOOTSTRAPS)

print("Computing CIs on discretized yhat binarized performance compared to KLG")
analysis.bootstrap_CIs_on_model_performance(y=binarized_y,
                                            yhat=discretized_yhat, 
                                            yhat_from_klg=klg, 
                                            yhat_from_clinical_image_features=None,
                                            ids=ids, 
                                            binary_prediction=True,
                                            n_bootstraps=N_BOOTSTRAPS)
                                            



In [ ]:
analysis.make_violin_nonredundancy_plot(yhat=yhat, klg=klg)

# Make simple histogram for presentation. Checked. 

In [ ]:
analysis.make_simple_histogram_of_pain(y=y, 
                              binary_vector_to_use=race_black, 
                              positive_class_label='Black patients', 
                              negative_class_label='Other patients', 
                              plot_filename='simple_pain_histogram_by_race.pdf')

# Robustness check: confirm results look similar when using womac pain score rather than Koos pain score. Checked. 

In [ ]:
print("Correlation between Womac and Koos subscores on test dataset: %2.3f" % 
      pearsonr(datasets['train'].non_image_data['koos_pain_subscore'].values,
               datasets['train'].non_image_data['womac_pain_subscore'].values)[0])

womac_yhat_from_klg = analysis.compare_to_clinical_performance(train_df=datasets['train'].non_image_data, 
                                val_df=datasets['val'].non_image_data, 
                                test_df=datasets['test'].non_image_data, 
                                y_col='womac_pain_subscore', 
                                features_to_use=['C(xrkl)'], 
                                binary_prediction=False, 
                                use_nonlinear_model=False, 
                                do_ols_sanity_check=True)

womac_y = datasets['test'].non_image_data['womac_pain_subscore'].values

print("Comparing performance (just in terms of R^2)") # because yhat and womac_y aren't on same scale, so can't do RMSE. 
print('KLG R^2 for predicting Womac pain score: %2.3f' % pearsonr(womac_yhat_from_klg, womac_y)[0] ** 2)
print('yhat R^2 for predicting Womac pain score: %2.3f' % pearsonr(yhat, womac_y)[0] ** 2)

print('KLG SPEARMAN R^2 for predicting Womac pain score: %2.3f' % spearmanr(womac_yhat_from_klg, womac_y)[0] ** 2)
print('yhat SPEARMAN R^2 for predicting Womac pain score: %2.3f' % spearmanr(yhat, womac_y)[0] ** 2)

print("Comparing reductions in WOMAC pain gap")
analysis.quantify_pain_gap_reduction_vs_rival(yhat=yhat, 
                                              y=womac_y, 
                                              rival_severity_measure=womac_yhat_from_klg, 
                                              all_ses_vars=all_ses_vars, 
                                             ids=ids)


In [ ]:
# BOOTSTRAPPED CIs for stratification plot. 

analysis.make_ses_stratification_plot(ses=graduated_college, 
                                     y=y, 
                                     dict_of_severity_scores={'KLG':klg, 
                                                              '$\hat y$ decile':decile_yhat},
                                    severity_score_order=['KLG', 
                                                          '$\hat y$ decile'], 
                                      ses_var_one_label='College grad', 
                                      ses_var_zero_label='Not college grad', 
                                      fig_title='education_pain_gap.png', 
                                      n_bootstraps=N_BOOTSTRAPS, 
                                      ids=ids)

analysis.make_ses_stratification_plot(ses=~(race_black == 1), 
                                     y=y, 
                                     dict_of_severity_scores={'KLG':klg, 
                                                              '$\hat y$ decile':decile_yhat},
                                    severity_score_order=['KLG', 
                                                          '$\hat y$ decile'], 
                                      ses_var_one_label='Other patients', 
                                      ses_var_zero_label='Black patients', 
                                      fig_title='black_pain_gap.png', 
                                      n_bootstraps=N_BOOTSTRAPS, 
                                      ids=ids)

analysis.make_ses_stratification_plot(ses=income_at_least_50k, 
                                     y=y, 
                                     dict_of_severity_scores={'KLG':klg, 
                                                              '$\hat y$ decile':decile_yhat},
                                    severity_score_order=['KLG', 
                                                          '$\hat y$ decile'], 
                                      ses_var_one_label='Income >= 50k', 
                                      ses_var_zero_label='Income < 50k', 
                                      fig_title='income_pain_gap.png', 
                                      ids=ids, 
                                      n_bootstraps=N_BOOTSTRAPS)

In [ ]:
# compare_two_severity_scores_on_one_plot. 

analysis.make_ses_stratification_plot(ses=~(race_black == 1), 
                                     y=y, 
                                     dict_of_severity_scores={'KLG':klg, 
                                                              'Discretized yhat':discretized_yhat},
                                    severity_score_order=['KLG', 
                                                          'Discretized yhat'], 
                                      ses_var_one_label='Other patients', 
                                      ses_var_zero_label='Black patients', 
                                      compare_two_severity_scores_on_one_plot=True)

analysis.make_ses_stratification_plot(ses=graduated_college, 
                                     y=y, 
                                     dict_of_severity_scores={'KLG':klg, 
                                                              'Discretized yhat':discretized_yhat},
                                    severity_score_order=['KLG', 
                                                          'Discretized yhat'], 
                                      ses_var_one_label='College grad', 
                                      ses_var_zero_label='Not college grad', 
                                      compare_two_severity_scores_on_one_plot=True)

analysis.make_ses_stratification_plot(ses=income_at_least_50k, 
                                     y=y, 
                                     dict_of_severity_scores={'KLG':klg, 
                                                              'Discretized yhat':discretized_yhat},
                                    severity_score_order=['KLG', 
                                                          'Discretized yhat'], 
                                      ses_var_one_label='Income >= 50k', 
                                      ses_var_zero_label='Income < 50k', 
                                      compare_two_severity_scores_on_one_plot=True)

In [ ]:
# Basically, before, we were fitting a predictive model to map KLG -> y, 
# then using that same model to map discretized_yhat -> y. 
# But this seems quite unfair to discretized_yhat because mean(y) might not be the same for the 5 groups. 
# So now instead we fit a linear model directly on discretized_yhat. 
# The only problem is, we only have that for the test set, which isn't ideal. 
# Ideally we would fit on the val set. I don't think this is a big deal, because overfitting should be minimal 
# (very few parameters to fit) and we outperform KLG regardless. 

discretized_yhat_df = pd.DataFrame({'koos_pain_subscore':datasets['test'].non_image_data['koos_pain_subscore'].values,
                                    'xrkl':discretized_yhat})

fit_discretized_yhat_model = sm.OLS.from_formula('koos_pain_subscore ~ C(discretized_yhat)', 
                                                data=discretized_yhat_df,
                                                ).fit()

pain_prediction_from_discretized_yhat = fit_discretized_yhat_model.predict(discretized_yhat_df).values

In [ ]:
# Predict from max of joint space narrowing. 

for dataset in ['train', 'val', 'test']:
    assert 'joint_space_narrowing_max' not in datasets[dataset].non_image_data.columns
    datasets[dataset].non_image_data['joint_space_narrowing_max'] = np.maximum(datasets[dataset].non_image_data['xrjsm'].values,
                                                                               datasets[dataset].non_image_data['xrjsl'].values
                                                                              )
    print("Proportion of JSN values in %s dataset" % dataset)
    print(100 * datasets[dataset].non_image_data['joint_space_narrowing_max'].value_counts()/len(datasets[dataset].non_image_data))

In [ ]:
# rival predictors. 
print("\n\n****predicting y from KLG")
yhat_from_klg = analysis.compare_to_clinical_performance(train_df=datasets['train'].non_image_data, 
                                val_df=datasets['val'].non_image_data, 
                                test_df=datasets['test'].non_image_data, 
                                y_col='koos_pain_subscore', 
                                features_to_use=['C(xrkl)'], 
                                binary_prediction=False, 
                                use_nonlinear_model=False, 
                                do_ols_sanity_check=True)

print("\n\n****predicting y from JSN max")
yhat_from_joint_space_narrowing_max = analysis.compare_to_clinical_performance(train_df=datasets['train'].non_image_data, 
                                val_df=datasets['val'].non_image_data, 
                                test_df=datasets['test'].non_image_data, 
                                y_col='koos_pain_subscore', 
                                features_to_use=['C(joint_space_narrowing_max)'], 
                                binary_prediction=False, 
                                use_nonlinear_model=False, 
                                do_ols_sanity_check=True)

print("\n\n****predicting y from LINEAR MODEL using all clinical image features (including KLG)")
linear_yhat_from_clinical_image_features = analysis.compare_to_clinical_performance(train_df=datasets['train'].non_image_data, 
                                val_df=datasets['val'].non_image_data, 
                                test_df=datasets['test'].non_image_data, 
                                y_col='koos_pain_subscore', 
                                features_to_use=['C(%s)' % a for a in CLINICAL_CONTROL_COLUMNS], 
                                binary_prediction=False,
                                use_nonlinear_model=False, 
                                do_ols_sanity_check=True)                                                   

nonlinear_interactions = []
for c1 in CLINICAL_CONTROL_COLUMNS:
    for c2 in CLINICAL_CONTROL_COLUMNS:
        if c1 > c2:
            nonlinear_interactions.append('C(%s) * C(%s)' % (c1, c2))

print("\n\n****predicting y from NONLINEAR MODEL using all clinical image features (including KLG)")
nonlinear_yhat_from_clinical_image_features = analysis.compare_to_clinical_performance(train_df=datasets['train'].non_image_data, 
                                val_df=datasets['val'].non_image_data, 
                                test_df=datasets['test'].non_image_data, 
                                y_col='koos_pain_subscore', 
                                features_to_use=nonlinear_interactions, 
                                binary_prediction=False,
                                use_nonlinear_model=False, 
                                do_ols_sanity_check=False)

print("\n\n****predicting y using RANDOM FOREST MODEL using all clinical image features (including KLG)")
random_forest_yhat_from_clinical_image_features = analysis.compare_to_clinical_performance(train_df=datasets['train'].non_image_data, 
                                val_df=datasets['val'].non_image_data, 
                                test_df=datasets['test'].non_image_data, 
                                y_col='koos_pain_subscore', 
                                features_to_use=['C(%s)' % a for a in CLINICAL_CONTROL_COLUMNS], 
                                binary_prediction=False,
                                use_nonlinear_model=True, 
                                do_ols_sanity_check=False)




# Predict from MRI features.

In [ ]:
original_non_image_data_for_filtering_mri = non_image_data_processing.NonImageData('all',
    i_promise_i_really_want_to_use_the_blinded_hold_out_set=True,                                                        
    timepoints_to_filter_for=['12 month follow-up', 
                              '24 month follow-up', 
                              '36 month follow-up', 
                              '48 month follow-up', 
                              '00 month follow-up: Baseline'], 
                             filter_out_special_values_in_mri_data=True)

In [ ]:
# Do this analysis totally separately (for pain gap as well) 
# because it's on a different, narrower subset (since most people don't have MRIs)
# As an additional robustness check, we repeat the analysis filtering out special values in MRI data just because 
# I'm not exactly sure the binarization procedure makes sense for these guys. 

df_for_filtering = original_non_image_data_for_filtering_mri.processed_dataframes['david_mri_data'][['id', 'side', 'visit']].copy()
df_for_filtering['no_special_values'] = True

mri_features = ['car11plusm', 'car11plusl', 'car11pluspf','bml2plusm', 'bml2plusl', 'bml2pluspf','mentearm', 'mentearl', 'menextm', 'menextl']

for k in mri_features:
    print("MRI feature %s" % k)
    print("Values are")
    print(datasets['train'].non_image_data[k].value_counts(dropna=False)/len(datasets['train']))

for also_include_xray_features in [False, True]:
    for use_random_forest in [False, True]:
        for df_for_filtering_out_special_values in [None, df_for_filtering]:
            analysis.compare_to_mri_features(datasets=datasets, 
                                 y=y, 
                                 yhat=yhat, 
                                 all_ses_vars=all_ses_vars, 
                                 ids=ids,
                                 df_for_filtering_out_special_values=df_for_filtering_out_special_values, 
                                also_include_xray_features=also_include_xray_features, 
                                 use_random_forest=use_random_forest, 
                                mri_features=mri_features)

# Robustness check -- how do clinical image features do on KLG >= 2. (We don't fill in missing data for KLG >= 2, so these features are the true values.) This is just a robustness check to make sure that it isn't our adding noise to the features which is causing our superior performance. 

In [ ]:
for use_nonlinear_model in [False, True]:
    print("\n\n\n****Assessing image feature performance on KLG>=2 with nonlinear model=%s" % use_nonlinear_model)
    rival_predictor_just_on_klg_geq_2 = analysis.compare_to_clinical_performance(
        train_df=datasets['train'].non_image_data.loc[datasets['train'].non_image_data['xrkl'] >= 2],
        val_df=datasets['val'].non_image_data.loc[datasets['val'].non_image_data['xrkl'] >= 2],
        test_df=datasets['test'].non_image_data.loc[datasets['test'].non_image_data['xrkl'] >= 2], 
                                    y_col='koos_pain_subscore', 
                                    features_to_use=['C(%s)' % a for a in CLINICAL_CONTROL_COLUMNS], 
                                    binary_prediction=False,
                                    use_nonlinear_model=use_nonlinear_model, 
                                    do_ols_sanity_check=True) 
    
    klg_geq_2_idxs = (datasets['test'].non_image_data['xrkl'] >= 2).values
    print("\nIn contrast, our performance is")
    our_performance_on_klg_geq_2 = analysis.assess_performance(y=y[klg_geq_2_idxs], 
                                yhat=yhat[klg_geq_2_idxs], 
                                binary_prediction=False)
    for k in our_performance_on_klg_geq_2:
        print("%s: %2.3f" % (k, our_performance_on_klg_geq_2[k]))
    
    print("Do we still reduce the pain gap more?")
    all_ses_vars_just_on_klg_geq_2 = {}
    for var in all_ses_vars:
        all_ses_vars_just_on_klg_geq_2[var] = all_ses_vars[var][klg_geq_2_idxs]

    print(analysis.quantify_pain_gap_reduction_vs_rival(yhat=yhat[klg_geq_2_idxs], 
                                              y=y[klg_geq_2_idxs], 
                                              rival_severity_measure=rival_predictor_just_on_klg_geq_2, 
                                              all_ses_vars=all_ses_vars_just_on_klg_geq_2, 
                                             ids=ids[klg_geq_2_idxs]))





# Small sanity check. ses gap looks roughly the same regardless of whether you do y ~ ses + C(klg) or whether you first predict y from klg and do y ~ ses + yhat_from_klg. 


In [ ]:
small_sanity_check_df = pd.DataFrame({'klg':klg, 'y':y, 'yhat_from_klg':yhat_from_klg})

# make sure r^2 is the same.
just_compute_r2_david_method = sm.OLS.from_formula('y ~ C(klg)', data=small_sanity_check_df).fit().rsquared
just_compute_r2_paper_method = sm.OLS.from_formula('y ~ yhat_from_klg', data=small_sanity_check_df).fit().rsquared
rel_diff = 100 * np.abs((just_compute_r2_david_method - just_compute_r2_paper_method) / just_compute_r2_paper_method)
print("R^2 for y ~ C(KLG) is %2.6f as compared to %2.6f in paper; rel diff %2.1f%%" % (
    just_compute_r2_david_method, just_compute_r2_paper_method, rel_diff))
assert rel_diff < 1

for ses_var in all_ses_vars:
    small_sanity_check_df['ses'] = all_ses_vars[ses_var] * 1.
    david_method = sm.OLS.from_formula('y ~ C(klg) + ses', data=small_sanity_check_df).fit()
    paper_method = sm.OLS.from_formula('y ~ yhat_from_klg + ses', data=small_sanity_check_df).fit()
    rel_diff = 100 * np.abs((david_method.params['ses'] - paper_method.params['ses']) / paper_method.params['ses'])
    print('%s coefficient with y ~ C(klg) + ses: %2.6f; %2.6f with method in paper; diff %2.1f%%' % (
        ses_var, david_method.params['ses'], paper_method.params['ses'], rel_diff))
    assert rel_diff < 1
    

# KLG performance stratified by SES. Note that these results are just on the test set and as such are noisy for small groups of people. 

In [ ]:
all_klg_performances_stratified_by_ses = []
for ses_var in all_ses_vars:
    for ses_var_val in [0, 1]:
        ses_idxs = all_ses_vars[ses_var] == ses_var_val
        klg_performance_ses_subgroup = analysis.assess_performance(y=y[ses_idxs], 
                                                        yhat=yhat_from_klg[ses_idxs], 
                                                        binary_prediction=False)
        klg_performance_ses_subgroup['klg mean'] = klg[ses_idxs].mean()
        klg_performance_ses_subgroup['klg std'] = klg[ses_idxs].std(ddof=1)
        klg_performance_ses_subgroup['pain mean'] = y[ses_idxs].mean()
        klg_performance_ses_subgroup['pain std'] = y[ses_idxs].std(ddof=1)
        klg_performance_ses_subgroup['Subset'] = '%s=%i' % (ses_var, ses_var_val)
        del klg_performance_ses_subgroup['negative_rmse']
        all_klg_performances_stratified_by_ses.append(klg_performance_ses_subgroup)
pd.DataFrame(all_klg_performances_stratified_by_ses)

    
    

# Pain gaps controlling for all three variables at once. Note that this is run on the full dataset, so results will not quite line up with stuff run only on the test dataset. 

In [ ]:

from statsmodels.iolib.summary2 import summary_col
df_for_regression = pd.concat([datasets['train'].non_image_data, 
                              datasets['val'].non_image_data, 
                              datasets['test'].non_image_data])
pain_gaps_controlling_for_all_three_at_once = []
for specification in ['binarized_income_at_least_50k', 
                      'binarized_education_graduated_college', 
                      'race_black', 
                      'binarized_income_at_least_50k + binarized_education_graduated_college + race_black', 
                      'binarized_income_at_least_50k + C(xrkl)', 
                      'binarized_education_graduated_college + C(xrkl)',
                      'race_black + C(xrkl)',
                      'binarized_income_at_least_50k + binarized_education_graduated_college + race_black + C(xrkl)']:
                      
                    
                      
    all_ses_vars_model = sm.OLS.from_formula('koos_pain_subscore ~ %s' % specification, data=df_for_regression).fit()
    all_ses_vars_model = all_ses_vars_model.get_robustcov_results(cov_type='cluster', 
                                                              groups=df_for_regression['id'].astype(int))
    pain_gaps_controlling_for_all_three_at_once.append(all_ses_vars_model)
summary_col(pain_gaps_controlling_for_all_three_at_once, 
            stars=True,
            model_names=range(len(pain_gaps_controlling_for_all_three_at_once)),
            regressor_order=['race_black', 'binarized_income_at_least_50k', 'binarized_education_graduated_college'])



# Show we get slightly better performance. 

In [ ]:

yhat_performance = analysis.assess_performance(y=y, yhat=yhat, binary_prediction=False)
discretized_yhat_performance = analysis.assess_performance(y=y, yhat=pain_prediction_from_discretized_yhat, binary_prediction=False)
klg_performance = analysis.assess_performance(y=y, yhat=yhat_from_klg, binary_prediction=False)
joint_space_narrowing_max_performance = analysis.assess_performance(y=y, 
                                                                   yhat=yhat_from_joint_space_narrowing_max, 
                                                                   binary_prediction=False)
all_clinical_performance_linear = analysis.assess_performance(y=y, 
                                                       yhat=linear_yhat_from_clinical_image_features, 
                                                       binary_prediction=False)
all_clinical_performance_nonlinear = analysis.assess_performance(y=y, 
                                                       yhat=nonlinear_yhat_from_clinical_image_features, 
                                                       binary_prediction=False)

all_clinical_performance_random_forest = analysis.assess_performance(y=y, 
                                                       yhat=random_forest_yhat_from_clinical_image_features, 
                                                       binary_prediction=False)


yhat_performance['predictor'] = 'yhat'
klg_performance['predictor'] = 'C(klg)'
joint_space_narrowing_max_performance['predictor'] = 'C(joint_space_narrowing_max)'
discretized_yhat_performance['predictor'] = 'discretized yhat'
all_clinical_performance_linear['predictor'] = 'all clinical linear'
all_clinical_performance_nonlinear['predictor'] = 'all clinical nonlinear'
all_clinical_performance_random_forest['predictor'] = 'all clinical random forest'

combined_results_df = pd.DataFrame([klg_performance, 
                    all_clinical_performance_linear, 
                    all_clinical_performance_nonlinear,
                    all_clinical_performance_random_forest,
                    joint_space_narrowing_max_performance,
                    discretized_yhat_performance,
                    yhat_performance])[['predictor', 'rmse', 'r', 'spearman_r', 'r^2', 'spearman_r^2']]

combined_results_df



In [ ]:
# CIs. 

analysis.bootstrap_CIs_on_model_performance(y=y,
                                            yhat=yhat, 
                                            yhat_from_klg=yhat_from_klg, 
                                            yhat_from_clinical_image_features=linear_yhat_from_clinical_image_features,
                                            ids=ids, 
                                            n_bootstraps=N_BOOTSTRAPS)



In [ ]:
# Write out DF for Ziad.

df_for_ziad = pd.DataFrame({'patient_id':ids, 
                            'side':datasets['test'].non_image_data['side'].values, 
                            'timepoint':datasets['test'].non_image_data['visit'].values,
                            'income_at_least_50k':income_at_least_50k, 
                            'graduated_college':graduated_college, 
                            'race_black':race_black,
                            'koos_pain_subscore':y, 
                            'klg':klg, 
                            'klg_p':yhat_from_klg, 
                            'alg_p':yhat, 
                            'discretized_alg_p':discretized_yhat
                           })

df_for_ziad = df_for_ziad[['patient_id', 'side', 'timepoint', 
                           'income_at_least_50k', 'graduated_college', 'race_black',
                           'klg', 'klg_p', 
                           'koos_pain_subscore','alg_p', 'discretized_alg_p']] 
assert pd.isnull(df_for_ziad).values.sum() == 0
assert Counter(df_for_ziad['discretized_alg_p']) == Counter(df_for_ziad['klg'])
assert len(df_for_ziad) == 11320
assert len(df_for_ziad.drop_duplicates(['patient_id', 'timepoint', 'side'])) == 11320
assert len(df_for_ziad.drop_duplicates('patient_id')) == 1295
assert len(df_for_ziad.drop_duplicates(['patient_id', 'graduated_college', 'income_at_least_50k', 'race_black'])) == 1295
assert len(df_for_ziad.drop_duplicates(['klg', 'klg_p'])) == 5
df_for_ziad.to_csv('df_for_ziad.csv', index=False)

# Show we reduce pain gap. 

In [ ]:
# compare to reduction from KLG. 
analysis.quantify_pain_gap_reduction_vs_rival(yhat=yhat, 
                                              y=y, 
                                              rival_severity_measure=yhat_from_klg, 
                                              all_ses_vars=all_ses_vars, 
                                             ids=ids)

# Omitted variable bias decomposition. 

### "Short equals long plus the effect of omitted times the regression of omitted on included."


In [ ]:

for ses_var in all_ses_vars:
    all_ovb_results = []
    print("\n\nSES var: %s" % ses_var)
    for severity_score in ['yhat', 'klg']:
        if severity_score == 'yhat':
            severity_score_to_use = yhat
        else:
            severity_score_to_use = yhat_from_klg
        severity_score_to_use = copy.deepcopy(severity_score_to_use)
        zscored_severity_score = (severity_score_to_use - severity_score_to_use.mean()) / severity_score_to_use.std(ddof=1)
        df_for_ovb_regression = pd.DataFrame({'koos_pain_subscore':y, 
                      'ses':all_ses_vars[ses_var]* 1., 
                      'severity_score':zscored_severity_score})

        short_reg = sm.OLS.from_formula('koos_pain_subscore ~ ses', data=df_for_ovb_regression).fit()
        long_reg = sm.OLS.from_formula('koos_pain_subscore ~ ses + severity_score', data=df_for_ovb_regression).fit()
        omitted_on_included = sm.OLS.from_formula('severity_score ~ ses', data=df_for_ovb_regression).fit()
        all_ovb_results.append({'reduction_in_pain_gap':short_reg.params['ses'] - long_reg.params['ses'], 
                                'effect_of_omitted':long_reg.params['severity_score'], 
                                'omitted_on_included':omitted_on_included.params['ses'], 
                                'severity_score':severity_score})
        assert np.allclose(short_reg.params['ses'] - long_reg.params['ses'], 
                           long_reg.params['severity_score'] * omitted_on_included.params['ses'])
    all_ovb_results = pd.DataFrame(all_ovb_results)
    all_ovb_results.index = all_ovb_results['severity_score']
    del all_ovb_results['severity_score']
    
    all_ovb_results.loc['RATIO'] = all_ovb_results.iloc[0] / all_ovb_results.iloc[1]
    print(all_ovb_results)
    

In [ ]:
# Bootstrap CIs on pain gap. 

analysis.bootstrap_CIs_on_pain_gap_reduction(y=y, 
                                             yhat=yhat, 
                                             yhat_from_klg=yhat_from_klg, 
                                             ids=ids, 
                                             all_ses_vars=all_ses_vars, 
                                             n_bootstraps=N_BOOTSTRAPS)

In [ ]:
# compare to reduction from using all clinical features (linear model)

analysis.quantify_pain_gap_reduction_vs_rival(yhat=yhat, 
                                              y=y, 
                                              rival_severity_measure=linear_yhat_from_clinical_image_features, 
                                              all_ses_vars=all_ses_vars, 
                                             ids=ids)

In [ ]:
# compare to reduction from using all clinical features (nonlinear model)
analysis.quantify_pain_gap_reduction_vs_rival(yhat=yhat, 
                                              y=y, 
                                              rival_severity_measure=nonlinear_yhat_from_clinical_image_features, 
                                              all_ses_vars=all_ses_vars, 
                                             ids=ids)

In [ ]:
# compare to reduction from using joint space narrowing max. 
analysis.quantify_pain_gap_reduction_vs_rival(yhat=yhat, 
                                              y=y, 
                                              rival_severity_measure=yhat_from_joint_space_narrowing_max, 
                                              all_ses_vars=all_ses_vars, 
                                             ids=ids)

In [ ]:
# compare to reduction from using all clincial features + random forest (nonlinear model)
analysis.quantify_pain_gap_reduction_vs_rival(yhat=yhat, 
                                              y=y, 
                                              rival_severity_measure=random_forest_yhat_from_clinical_image_features, 
                                              all_ses_vars=all_ses_vars, 
                                             ids=ids)



In [ ]:
# compare discretized yhat to reduction from KLG. 

analysis.quantify_pain_gap_reduction_vs_rival(yhat=pain_prediction_from_discretized_yhat, 
                                              y=y, 
                                              rival_severity_measure=yhat_from_klg, 
                                              all_ses_vars=all_ses_vars, 
                                             ids=ids)

# Check main results hold when controlling for things. 

In [ ]:
"""
From our email chain, covariates are:
Recruitment site
Age*sex
Timepoint
Side
Current BMI
Max BMI
Past knee injury
Past knee surgery
Current / former smoker
Drinking status
Marital status

Note: we sometimes get "condition number is large" warnings when we do regressions of y ~ yhat. 
This does not appear to indicate a convergence problem. Rather, it's just that y and yhat are on a large scale (0-100). 
If you just divide both variables by 100, the coefficients (of course) remain identical and the warnings go away. 
"""
all_comparisons = []
for rival_name in ['klg', 'all_clinical_features']:
    for control_set in [['C(v00site)', 
                         'C(age_at_visit)*C(p02sex)',
                         'C(visit)', 
                         'side', 
                          'C(max_bmi)', 
                         'C(current_bmi)',
                        'C(knee_surgery)', 
                         'C(knee_injury)', 
                         'C(cigarette_smoker)', 
                         'C(drinks_per_week)', 
                         'C(v00maritst)'] ,
                        ['C(age_at_visit)*C(p02sex)'], 
                        ['C(age_at_visit)*C(p02sex)', 'side', 'C(visit)', 'C(v00site)']]:
        if rival_name == 'klg':
            rival_predictor = yhat_from_klg
        else:
            rival_predictor = linear_yhat_from_clinical_image_features
        print("\n\n\n\n*************** COMPARING %s to yhat controlling for" % rival_name)
        print(control_set)
        comparison = analysis.check_main_results_hold_when_controlling_for_things(df=datasets['test'].non_image_data, 
                                                             yhat=yhat, 
                                                             rival_predictor=rival_predictor, 
                                                             rival_name=rival_name,
                                                             all_controls=control_set)
        all_comparisons.append(comparison)

pd.set_option('max_colwidth', 500)
pd.concat(all_comparisons)

# Additional check: we aren't predicting y merely by predicting SES. 

In [ ]:
print("Check yhat coefficient is still coefficient when controlling for xrkl and SES")

df_to_test = pd.DataFrame({'y':y, 
                               'yhat_from_klg':yhat_from_klg,
                               'yhat':yhat, 
                               'klg':klg,
                               'id':ids})
for ses_var in all_ses_vars:
    df_to_test[ses_var] = all_ses_vars[ses_var]
    print("SES var: %s" % ses_var)
    
    print('R^2 for y ~ yhat + SES: %2.3f; for y ~ KLG + ses: %2.3f' % 
          (
              sm.OLS.from_formula('y ~ yhat + %s' % ses_var, data=df_to_test).fit().rsquared, 
              sm.OLS.from_formula('y ~ yhat_from_klg + %s' % ses_var, data=df_to_test).fit().rsquared
          )
         )

    ols_model = sm.OLS.from_formula('y ~ yhat + %s + yhat_from_klg' % ses_var, data=df_to_test).fit(
        cov_type='cluster', cov_kwds={'groups':df_to_test['id']})
    print('y ~ yhat + SES + KLG: yhat coefficient %2.3f, p=%2.3e' % (ols_model.params['yhat'], ols_model.pvalues['yhat']))

print("yhat Coefficient when not controlling for any SES vars")
control_for_no_ses_vars = sm.OLS.from_formula('y ~ yhat', data=df_to_test).fit(cov_type='cluster', cov_kwds={'groups':df_to_test['id']})
print(control_for_no_ses_vars.summary())
print("yhat Coefficient when not controlling for all SES vars")
control_for_all_three_ses_vars = sm.OLS.from_formula('y ~ yhat + race_black + income_less_than_50k + did_not_graduate_college', 
                                                     data=df_to_test).fit(cov_type='cluster', cov_kwds={'groups':df_to_test['id']})
print(control_for_all_three_ses_vars.summary())

# Relatedly what is coefficient of KLG when controlling for yhat (don't need this in for loop because not using SES var)
print("**\n\nIs KLG significant when controlling for yhat")

ols_model = sm.OLS.from_formula('y ~ yhat + yhat_from_klg', data=df_to_test).fit(
    cov_type='cluster', cov_kwds={'groups':df_to_test['id']})
print(ols_model.summary())

# Try doing an f-test with categorical dummies. 
# Note that an f-test is not valid here when we cluster; there's no way to account for the clustering. 
# So I don't think the f-test is the way to do it. 
restricted_model_unclustered = sm.OLS.from_formula('y ~ yhat', data=df_to_test).fit()
full_model_unclustered = sm.OLS.from_formula('y ~ yhat + C(klg)', data=df_to_test).fit()

print(full_model_unclustered.compare_f_test(restricted_model_unclustered))


In [ ]:
# another check: check that we outperform across a whole bunch of subgroups. 
analysis.stratify_performances(df=datasets['test'].non_image_data, 
                               yhat=yhat, 
                               y=y, 
                               yhat_from_klg=yhat_from_klg)

# Robustness check -- do we generalize across sites?

In [ ]:
all_site_generalization_results = analysis.load_all_results(binary=False, 
                                        min_timestring='2019_06_20', 
                                        thing_to_filter_by={'experiment_to_run':'hold_out_one_imaging_site'})

all_site_generalization_results['site_to_remove'] = all_site_generalization_results['hold_out_one_imaging_site_kwargs'].map(
    lambda x:x['site_to_remove'])


In [ ]:
# sanity check: does computing the val loss on the held out site make a difference in how models are ranked
# either in terms of choosing the best epoch or the best model? 
# doesn't seem to, no. 

best_results_by_model = []
for i in range(len(all_site_generalization_results)):
    # first compare within models. 
    val_results_for_model = all_site_generalization_results['val_results_stratified_by_site'].iloc[i]
    site_excluded = all_site_generalization_results['site_to_remove'].iloc[i]
    epochs = sorted(val_results_for_model.keys())
    assert epochs == list(range(15))
    site_excluded_results = [val_results_for_model[a]['every_site_but_%s' % site_excluded]['negative_rmse']
                             for a in epochs]
    full_val_set_results = [val_results_for_model[a]['val_negative_rmse']
                             for a in epochs]
    best_results_by_model.append({'site_excluded_results':max(site_excluded_results), 
                                  'full_val_set_results':max(full_val_set_results), 
                                  'site_excluded':site_excluded})
    plt.figure()
    plt.title("Each point is one EPOCH for a single model site %s\nspearman r %2.3f" % 
              (site_excluded, spearmanr(full_val_set_results, site_excluded_results)[0]))
    plt.xlabel("Full val set RMSE")
    plt.ylabel("RMSE excluding site %s" % site_excluded)
    plt.scatter(full_val_set_results, site_excluded_results)

# then compare across models. 
best_results_by_model = pd.DataFrame(best_results_by_model)
for site_excluded in sorted(list(set(best_results_by_model['site_excluded']))):
    plt.figure()
    plt.title("Each point is one MODEL for site %s" % site_excluded)
    plt.scatter(best_results_by_model.loc[best_results_by_model['site_excluded'] == site_excluded, 'full_val_set_results'],
                best_results_by_model.loc[best_results_by_model['site_excluded'] == site_excluded, 'site_excluded_results'])
    plt.xlabel("Full val set RMSE")
    plt.ylabel("RMSE excluding site %s" % site_excluded)
    plt.show()
    
assert np.allclose(list(best_results_by_model['full_val_set_results'].values), 
            sorted(best_results_by_model['full_val_set_results'].values)[::-1])
assert((all_site_generalization_results['best_val_negative_rmse'] 
       == best_results_by_model['full_val_set_results'].values).all())
all_site_generalization_results['exclude_held_out_site_best_val_negative_rmse'] = best_results_by_model['site_excluded_results'].values


In [ ]:
# We win consistently on pain gap reduction across all sites. 
# r^2 and spearman r^2 are generally better. 
# But yhat is somewhat miscalibrated for the new site (r^2 is good, but RMSE is less so). 
# You can fix this by recalibrating both yhat and KLG for each site
# Recalibrating does not affect pain gap reduction or r^2. 
# Recalibrating seems a bit sketchy, though, so we don't do it in the results we report. 

key_to_sort_by = 'best_val_negative_rmse'#'exclude_held_out_site_best_val_negative_rmse' # 'best_val_negative_rmse'
all_site_generalization_results = all_site_generalization_results.sort_values(by=key_to_sort_by)[::-1]

for recalibrate_to_new_set in [True, False]:
    print("\n\n***********Recalibrating to new set: %s" % recalibrate_to_new_set)
    analysis.analyze_performance_on_held_out_sites(all_site_generalization_results, 
                                               yhat=yhat, 
                                               y=y, 
                                               yhat_from_klg=yhat_from_klg, 
                                               site_vector=datasets['test'].non_image_data['v00site'].values, 
                                               all_ses_vars=all_ses_vars, 
                                               ids=ids, 
                                               recalibrate_to_new_set=recalibrate_to_new_set)

# Robustness check: dataset where there is no correlation between pain and race/SES. I ultimately favored this robustness check over the one where we just train one race/SES group because that hurts predictive performance for reasons unrelated to what we're trying to assess (ie, we halve the training set size). (This robustness check does not end up in the paper.) 

In [ ]:
all_ses_decorrelated_results = analysis.load_all_results(binary=False, 
                                        min_timestring='2019_06_20', 
                                        thing_to_filter_by={'experiment_to_run':'remove_correlation_between_pain_and_ses'})

pd.set_option('precision', 3)
all_ses_decorrelated_results['ses_decorrelation_col'] = all_ses_decorrelated_results['remove_correlation_between_pain_and_ses_kwargs'].map(lambda x:x['ses_col'])
all_ses_decorrelated_results[['ses_decorrelation_col', 'test_r', 'negative_test_rmse']]

In [ ]:
# Confirm that we still narrow the pain gap more than KLG and we still have better predictive performance than KLG. 
all_performances = []
for ses_col in sorted(list(set(all_ses_decorrelated_results['ses_decorrelation_col']))):
    print('\n\n***Robustness check for dataset where pain and %s are decorrelated' % ses_col)
    _, ses_decorrelated_yhat = analysis.try_ensembling(
        all_ses_decorrelated_results.loc[all_ses_decorrelated_results['ses_decorrelation_col'] == ses_col],
        5,
        binary_prediction=False)
    performance = analysis.assess_performance(y=y, 
                                              yhat=ses_decorrelated_yhat, 
                                              binary_prediction=False)
    performance['ses_decorrelation_col'] = ses_col
    all_performances.append(performance)
    print("Pain gap reduction")
    print(analysis.quantify_pain_gap_reduction_vs_rival(yhat=ses_decorrelated_yhat, 
                                              y=y, 
                                              rival_severity_measure=yhat_from_klg, 
                                              all_ses_vars=all_ses_vars, 
                                             ids=ids))
    
print("All performance")
klg_performance = analysis.assess_performance(y=y, yhat=yhat_from_klg, binary_prediction=False)
klg_performance['ses_decorrelation_col'] = 'KLG BASELINE'
all_performances.append(klg_performance)
pd.DataFrame(all_performances)[['ses_decorrelation_col', 'rmse', 'r', 'spearman_r', 'r^2', 'spearman_r^2']]

# Followup predictions: does yhat help predict future KLG or pain? We try multiple specifications of this question. 

In [ ]:
pd.set_option("display.precision", 5)
pd.set_option('max_colwidth', 500)
analysis.predict_kl_at_future_timepoints(non_image_data=datasets['test'].non_image_data, yhat=yhat)
    
    

In [ ]:
# Compare to KLG.
analysis.predict_kl_at_future_timepoints(non_image_data=datasets['test'].non_image_data,
                                         yhat=yhat_from_klg)

In [ ]:
# effect of discretized_yhat=4 on continuous pain score + KLG. 

analysis.predict_kl_at_future_timepoints(non_image_data=datasets['test'].non_image_data,
                                         yhat=(discretized_yhat == 4) * 1.)

In [ ]:
# effect of discretized_yhat>=2 on continuous pain score + KLG. 

analysis.predict_kl_at_future_timepoints(non_image_data=datasets['test'].non_image_data,
                                         yhat=(discretized_yhat >= 2) * 1.)

In [ ]:
# effect of one std increase in yhat on odds of being high pain. 

analysis.predict_kl_at_future_timepoints(non_image_data=datasets['test'].non_image_data,
                                         yhat=-(yhat - yhat.mean()) / yhat.std(ddof=1), 
                                         use_binary_pain=True)

In [ ]:
# effect of discretized_yhat=4 on odds of being high pain. 

analysis.predict_kl_at_future_timepoints(non_image_data=datasets['test'].non_image_data,
                                         yhat=(discretized_yhat >= 4) * 1., 
                                         use_binary_pain=True)

In [ ]:
# effect of discretized_yhat>=2 on odds of being high pain. 

analysis.predict_kl_at_future_timepoints(non_image_data=datasets['test'].non_image_data,
                                         yhat=(discretized_yhat >= 2) * 1., 
                                         use_binary_pain=True)

In [ ]:
# again, yhat doesn't really have better predictive power for pain than KLG at followup when you control for pain at baseline.
# (although it does when you don't control for KLG). 
# This makes sense because yhat is more strongly correlated with pain than is KLG, so it's not clear it should add more than KLG should over pain.  
analysis.predict_kl_at_future_timepoints(non_image_data=datasets['test'].non_image_data,
                                         yhat=(klg >= 2) * 1., 
                                         use_binary_pain=True)

# Show we can predict KLG.

In [ ]:
# comparison to previous results:
# 1. Automatic Knee Osteoarthritis Diagnosis from Plain Radiographs: A Deep Learning-Based Approach
# Sophisticated model: manual cropping, siamese network etc. 
# We trained our method using the data solely from the Multicenter Osteoarthritis Study 
# and validated it on randomly selected 3,000 subjects (5,960 knees) from Osteoarthritis Initiative dataset (just baseline data)
# The classification MSE value achieved was 0.48, which is lower than previously published results...
# we also evaluated a fine-tuned ResNet-34 network because it performed similarly on the validation set. 
# On the test set, the baseline also performed similarly to our approach in terms of MSE (value of 0.51)
# More sophisticated model, probably a harder prediction task. 

# Quantifying Radiographic Knee Osteoarthritis Severity using Deep Convolutional Neural Networks
# MSE: 0.504
# The data used for the experiments are bilateral PA fixed flexion knee X-ray images, 
# taken from the baseline (image release version O.E.1) radiographs of the Osteoarthritis Initiative (OAI) dataset 
# containing an entire cohort of 4,476 participants

all_klg_results = analysis.load_all_results(binary=False, 
                                        min_timestring='2019_06_20', 
                                        thing_to_filter_by={'experiment_to_run':'predict_klg', 
                                                             'crop_to_just_the_knee':False})

ensemble_results_klg_prediction, ensemble_prediction_klg_hat = analysis.try_ensembling(all_klg_results, 5, binary_prediction=False)

pd.set_option('precision', 3)
ensemble_results_klg_prediction['MSE'] = ensemble_results_klg_prediction['negative_rmse'] ** 2
ensemble_results_klg_prediction



In [ ]:
# compare to baseline visit to get more precise comparison to previous work (which used baseline visit). 
true_klg = all_klg_results.iloc[0]['test_y']
baseline_idxs = (datasets['test'].non_image_data['visit'].map(lambda x:'Baseline' in x).values)
klg_prediction_results = analysis.assess_performance(y=true_klg[baseline_idxs],
                            yhat=ensemble_prediction_klg_hat[baseline_idxs], 
                            binary_prediction=False)

print("MSE just on baseline visits: %2.3f; r %2.3f" % (klg_prediction_results['negative_rmse'] ** 2, 
                                                       klg_prediction_results['r']))

In [ ]:
# how well does this predicted KLG predict pain?

print("Assessing how KLG_hat does compared to KLG")
# Note this might be a little generous to KLG_hat, because we just assess fit on the test set 
# but I don't think the problem should be very big because the number of free parameters is very small. 
# (The reason we do this is that we lack a train set KLG_hat to fit on, due to overfitting.) 

discretized_klg_hat = analysis.discretize_yhat_like_kl_grade(
    yhat_arr=-ensemble_prediction_klg_hat,
    kl_grade_arr=klg,
    y_col='koos_pain_subscore')

print(pd.DataFrame({'klg':klg,'discretized_klg_hat':discretized_klg_hat})
      .groupby(['klg', 'discretized_klg_hat'])
      .size()/len(klg))

klg_to_predict_pain_df = pd.DataFrame({'klg_hat':ensemble_prediction_klg_hat, 
                                      'y':y, 
                                       'discretized_klg_hat':discretized_klg_hat})

pain_predicted_from_klg_hat_model = sm.OLS.from_formula('y ~ klg_hat', data=klg_to_predict_pain_df).fit()
pain_predicted_from_discretized_klg_hat_model = sm.OLS.from_formula('y ~ C(discretized_klg_hat)', data=klg_to_predict_pain_df).fit()

all_predictors_to_compare = {'klg':yhat_from_klg, 
                             'klg_hat':pain_predicted_from_klg_hat_model.predict(klg_to_predict_pain_df).values, 
                             'discretized_klg_hat':pain_predicted_from_discretized_klg_hat_model.predict(klg_to_predict_pain_df).values}
all_klg_versions_performance = []
for predictor_name in ['klg', 'klg_hat', 'discretized_klg_hat']:
    predictor_performance = analysis.assess_performance(yhat=all_predictors_to_compare[predictor_name], 
                                                        y=y, 
                                                        binary_prediction=False)
    predictor_performance['predictor'] = predictor_name
    all_klg_versions_performance.append(predictor_performance)
print(pd.DataFrame(all_klg_versions_performance)[['predictor', 'r^2', 'spearman_r^2', 'rmse']])
    

for predictor_name in ['klg', 'klg_hat', 'discretized_klg_hat']:
    print('pain gap reduction due to %s' % predictor_name)
    print(analysis.quantify_pain_gap_reduction_vs_rival(yhat=all_predictors_to_compare[predictor_name],
                                              y=y, 
                                              rival_severity_measure=yhat_from_klg, 
                                              all_ses_vars=all_ses_vars, 
                                             ids=ids)[['SES var', 'yhat/rival red. ratio']])


                            

# Predict at future timepoints. 

In [ ]:
analysis.predict_kl_at_future_timepoints(non_image_data=datasets['test'].non_image_data,
                                         yhat=yhat)

# Interpretability

In [ ]:
correlations = analysis.assess_what_image_features_y_and_yhat_correlate_with(
                non_image_data=datasets['test'].non_image_data, 
                y=y, 
                yhat=yhat)

# Interventions. 

In [ ]:
# New Ziad surgery analysis. 
baseline_idxs = datasets['test'].non_image_data['visit'].values == '00 month follow-up: Baseline'
print("Showing that when we allocate surgery on the basis of pain + KLG, we give more to black patients")
analysis.do_surgery_analysis_ziad_style(yhat=yhat,
                                        y=y,
                                        klg=klg,
                                        all_ses_vars=all_ses_vars, 
                                        baseline_idxs=baseline_idxs,
                                        have_actually_had_surgery=datasets['test'].non_image_data['knee_surgery'].values, 
                                        df_to_use=datasets['test'].non_image_data, 
                                        ids=ids)

In [ ]:
# Show that disadvantaged groups have higher rates of surgery (both controlling and not controlling for KLG). 
concatenated_df_for_surgery_gap_analysis = pd.concat([datasets['train'].non_image_data, 
                                      datasets['val'].non_image_data, 
                                      datasets['test'].non_image_data])
concatenated_df_for_surgery_gap_analysis.index = range(len(concatenated_df_for_surgery_gap_analysis))

analysis.assess_treatment_gaps_controlling_for_klg(klg=concatenated_df_for_surgery_gap_analysis['xrkl'].values, 
            all_ses_vars = analysis.extract_all_ses_vars(concatenated_df_for_surgery_gap_analysis)[0], 
            baseline_idxs=concatenated_df_for_surgery_gap_analysis['visit'].values == '00 month follow-up: Baseline',
            df_to_use=concatenated_df_for_surgery_gap_analysis)

In [ ]:
analysis.make_painkillers_and_surgery_frequency_bar_plot(
    datasets['test'].non_image_data.loc[baseline_idxs])


In [ ]:
# New Ziad surgery analysis. This is just arthroscopies and shows no improvement in pain; 
# if anything, pain gets worse. This makes sense because the variable of real interest is arthroplasty, knee replacement
# so we bring that in by going back to the raw data. 
# This analysis does not end up in the final paper. 
original_non_image_data = non_image_data_processing.NonImageData('all',
    i_promise_i_really_want_to_use_the_blinded_hold_out_set=True,                                                        
    timepoints_to_filter_for=['12 month follow-up', 
                              '24 month follow-up', 
                              '36 month follow-up', 
                              '48 month follow-up', 
                              '00 month follow-up: Baseline'])

df_for_surgery_progression_analysis = pd.concat([datasets['train'].non_image_data, 
                                  datasets['val'].non_image_data, 
                                   datasets['test'].non_image_data])

old_len = len(df_for_surgery_progression_analysis)
df_for_surgery_progression_analysis = pd.merge(df_for_surgery_progression_analysis, 
                                              original_non_image_data.processed_dataframes['knee_replacement'], 
                                              how='inner', 
                                              on=['id', 'side', 'visit'])
assert len(df_for_surgery_progression_analysis) == old_len

# note that people get filtered out of our data if they have knee repalcement, so we can't even analyze this using our processed data. 
# (they get filtered out b/c KLG and other image features aren't meaningful)
print("Confirmation: the vast majority of points in our datapoint do not have knee replacement: proportion %2.6f do" % 
      df_for_surgery_progression_analysis['knee_replacement'].mean())


# Does pain decrease after general knee arthroscopy -- seems like answer is NO. 
print("Do people who get general surgery experience lower pain?")
analysis.study_effect_of_surgery(df_for_surgery_progression_analysis, 
                             surgery_col_to_analyze='knee_surgery')

In [ ]:
# Does pain decrease after arthroplasty. 
# Note this analysis cannot be done on our original dataset
# because all the people lack post surgery data, so we go back to the raw data. 
# This analysis does not end up in the final paper. 

pain_df = original_non_image_data.processed_dataframes['all_knee_pain_scores'].copy()
knee_replacement_df = original_non_image_data.processed_dataframes['knee_replacement'].copy()

merged_df = pd.merge(knee_replacement_df, pain_df, how='inner', on=['id', 'visit', 'side'], validate='one_to_one')
merged_df['high_pain'] = binarize_koos(merged_df['koos_pain_subscore'])
merged_df = merged_df.merge(original_non_image_data.processed_dataframes['medications'], 
                           how='inner', 
                           on=['id', 'visit'], validate='many_to_one')

test_prognostic_df = datasets['test'].non_image_data[['id', 'visit', 'side', 'xrkl']].copy()
test_prognostic_df['discretized_yhat'] = discretized_yhat
assert Counter(test_prognostic_df['discretized_yhat']) == Counter(test_prognostic_df['xrkl'])

merged_df = pd.merge(merged_df, test_prognostic_df, how='left', on=['id', 'visit', 'side'], validate='one_to_one')
analysis.study_effect_of_surgery(merged_df, 
                                 surgery_col_to_analyze='knee_replacement')

In [ ]:
# Rate of surgery increases with KLG. Note we compute this on full dataset. 

interventions_df = pd.concat([datasets['train'].non_image_data, 
                             datasets['val'].non_image_data, 
                             datasets['test'].non_image_data])
interventions_df = interventions_df.loc[interventions_df['visit'] == '00 month follow-up: Baseline']
interventions_df = interventions_df.dropna(subset=['knee_surgery'])
interventions_df.index = range(len(interventions_df))
analysis.make_rate_of_surgery_figure(interventions_df)

In [ ]:
analysis.make_painkillers_and_surgery_frequency_bar_plot(interventions_df)

In [ ]:
# Show assignment to higher risk categories and reassignment of surgery. 

baseline_idxs = datasets['test'].non_image_data['visit'].map(lambda x:x == '00 month follow-up: Baseline').values

analysis.make_scatter_plot_showing_severity_reassignment_under_yhat(yhat=yhat, 
                                                           y=y, 
                                                           klg=klg, 
                                                           all_ses_vars=all_ses_vars, 
                                                           idxs_to_use=baseline_idxs, 
                                                           interventions_df=interventions_df)


# What is the effect of diversity? 

In [ ]:
# Load in diversity results. 
all_diversity_results = analysis.load_all_results(binary=False,  min_timestring='2019_06_20', 
                                    thing_to_filter_by={'experiment_to_run':'increase_diversity', 
                                                         'crop_to_just_the_knee':False})

for k in ['exclude_minority_group', 'ses_col', 'majority_group_seed']:
    all_diversity_results[k] = all_diversity_results['increase_diversity_kwargs'].map(lambda x:x[k])
assert ((pd.isnull(all_diversity_results['majority_group_seed'])) == 
        (all_diversity_results['exclude_minority_group'])).all()
all_diversity_results['majority_group_seed'] = all_diversity_results['majority_group_seed'].map(str)


# analyze results. 
analysis.analyze_effect_of_diversity(all_diversity_results, 
                                     all_ses_vars=all_ses_vars,
                                     y=y, 
                                     yhat_from_klg=yhat_from_klg, 
                                     ids=ids, 
                                     n_bootstraps=N_BOOTSTRAPS)
                                     
                                     

# Not using code below here. 

In [ ]:
run_this_code = False
if run_this_code:
    raise Exception("Not using this code at present; in theory it computes the painkiller gap, but in practice it's pretty messy and you should check it if you use it.")
    ids_on_painkillers_at_baseline = set(medication_df.loc[medication_df['Narcotic_analgesic'] == 1, 'id'])

    painkiller_gap_df = datasets['test'].non_image_data[[
        'id',
        'visit',
        'binarized_education_graduated_college', 
        'binarized_income_at_least_50k', 
        'race_black', 
        'xrkl']].copy()
    painkiller_gap_df['discretized_yhat'] = discretized_yhat
    painkiller_gap_df['on_painkillers_at_baseline'] = 1.*painkiller_gap_df['id'].map(lambda x:x in ids_on_painkillers_at_baseline)
    painkiller_gap_df = painkiller_gap_df.loc[painkiller_gap_df['visit'].map(lambda x:'Baseline' in x)]

    for k in ['binarized_education_graduated_college', 
              'binarized_income_at_least_50k', 
              'race_black']:
        painkiller_model = sm.OLS.from_formula('on_painkillers_at_baseline ~ %s' % k, 
                                           data=painkiller_gap_df).fit()
        print(painkiller_model.params[k])
        painkiller_model = sm.OLS.from_formula('on_painkillers_at_baseline ~ %s + C(xrkl)' % k, 
                                           data=painkiller_gap_df).fit()
        print(painkiller_model.params[k])
        painkiller_model = sm.OLS.from_formula('on_painkillers_at_baseline ~ %s + C(discretized_yhat)' % k, 
                                           data=painkiller_gap_df).fit()
        print(painkiller_model.params[k])
                                                     